# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
import pandas as pd
import numpy as np
import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Error loading stopwords: HTTP Error 403: Forbidden


In [3]:
def load_data(dbfilepath, tablename):
    '''
    input: (
        dbfilepath: path to database
        tablename: name of table to fetch
            )
    Loads data from sqlite database 
    output: (
        X: features dataframe
        y: target dataframe
        category_names: names of targets
        )
    '''
    engine = create_engine(f'sqlite:///{dbfilepath}')
    df = pd.read_sql_table(tablename, engine)
    X = df['message']
    Y = df.iloc[:, 4:]
    category_names = Y.columns
    return X, Y, category_names

X, Y, category_names = load_data(dbfilepath='data\DisasterResponse.db', tablename='messages')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text, remove_stopwords=True):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    stopw = set(stopwords.words('english')) if remove_stopwords else []

    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens if token not in stopw]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect' , CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))      
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pred = pipeline.predict(X_test)

In [10]:
def display_results(Y_test, Y_pred, model=None):

    target_names = Y_test.columns.values
    
    # Calculate the accuracy for each of them.
    for i in range(len(category_names)):
        print(f'Category:, {category_names[i]} \n {classification_report(Y_test.iloc[:, i].values, Y_pred[:, i])}')
        print(f'Accuracy of {category_names[i]}: {accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i]):.2f}')
        
    if model:
        print(f'\nBest Parameters: {model.best_params_}')
                                
display_results(Y_test, Y_pred)

Category:, related 
               precision    recall  f1-score   support

           0       0.63      0.37      0.46      1873
           1       0.82      0.93      0.87      5934
           2       0.50      0.03      0.06        58

   micro avg       0.79      0.79      0.79      7865
   macro avg       0.65      0.44      0.47      7865
weighted avg       0.77      0.79      0.77      7865

Accuracy of related: 0.79
Category:, request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      6533
           1       0.81      0.40      0.54      1332

   micro avg       0.88      0.88      0.88      7865
   macro avg       0.85      0.69      0.73      7865
weighted avg       0.88      0.88      0.87      7865

Accuracy of request: 0.88
Category:, offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

   micro avg   

C:\Users\N12667\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Accuracy of medical_help: 0.92
Category:, medical_products 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      7447
           1       0.66      0.05      0.09       418

   micro avg       0.95      0.95      0.95      7865
   macro avg       0.80      0.52      0.53      7865
weighted avg       0.93      0.95      0.93      7865

Accuracy of medical_products: 0.95
Category:, search_and_rescue 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      7673
           1       0.65      0.14      0.22       192

   micro avg       0.98      0.98      0.98      7865
   macro avg       0.81      0.57      0.61      7865
weighted avg       0.97      0.98      0.97      7865

Accuracy of search_and_rescue: 0.98
Category:, security 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      7721
           1       0.00      0.00      0.00       1


Accuracy of weather_related: 0.85
Category:, floods 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      7242
           1       0.88      0.37      0.52       623

   micro avg       0.95      0.95      0.95      7865
   macro avg       0.92      0.68      0.75      7865
weighted avg       0.94      0.95      0.94      7865

Accuracy of floods: 0.95
Category:, storm 
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      7127
           1       0.73      0.46      0.57       738

   micro avg       0.93      0.93      0.93      7865
   macro avg       0.84      0.72      0.77      7865
weighted avg       0.93      0.93      0.93      7865

Accuracy of storm: 0.93
Category:, fire 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      7782
           1       0.00      0.00      0.00        83

   micro avg       0.99      0.99      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'tfidf__norm' : ['l2', None], 
        'clf__estimator__n_estimators': [50]
    }

model = GridSearchCV(pipeline, param_grid=parameters, cv=3)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
display_results(Y_test, Y_pred, model)

Category:, related 
               precision    recall  f1-score   support

           0       0.71      0.34      0.46      1873
           1       0.81      0.96      0.88      5934
           2       0.67      0.07      0.12        58

   micro avg       0.80      0.80      0.80      7865
   macro avg       0.73      0.45      0.49      7865
weighted avg       0.79      0.80      0.77      7865

Accuracy of related: 0.80
Category:, request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      6533
           1       0.84      0.47      0.61      1332

   micro avg       0.90      0.90      0.90      7865
   macro avg       0.87      0.73      0.77      7865
weighted avg       0.89      0.90      0.88      7865

Accuracy of request: 0.90
Category:, offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

   micro avg   

C:\Users\N12667\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Accuracy of aid_related: 0.77
Category:, medical_help 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96      7227
           1       0.72      0.07      0.13       638

   micro avg       0.92      0.92      0.92      7865
   macro avg       0.82      0.53      0.55      7865
weighted avg       0.91      0.92      0.89      7865

Accuracy of medical_help: 0.92
Category:, medical_products 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      7447
           1       0.88      0.10      0.18       418

   micro avg       0.95      0.95      0.95      7865
   macro avg       0.91      0.55      0.58      7865
weighted avg       0.95      0.95      0.93      7865

Accuracy of medical_products: 0.95
Category:, search_and_rescue 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      7673
           1       0.77      0.05      0.10       192

Category:, other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      7524
           1       0.00      0.00      0.00       341

   micro avg       0.96      0.96      0.96      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.92      0.96      0.94      7865

Accuracy of other_infrastructure: 0.96
Category:, weather_related 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      5702
           1       0.85      0.67      0.75      2163

   micro avg       0.88      0.88      0.88      7865
   macro avg       0.87      0.81      0.83      7865
weighted avg       0.87      0.88      0.87      7865

Accuracy of weather_related: 0.88
Category:, floods 
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      7242
           1       0.87      0.40      0.55       623

   micro avg       0.95   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [17]:
with open('models\model.pickle', 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.